In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet

In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236

In [8]:
import albumentations as albu

def get_train_augs(p=1.):
    return albu.Compose([
        #albu.HorizontalFlip(.5),
        albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5 ),
        albu.Blur(blur_limit=3, p=0.3),
        albu.OpticalDistortion(p=0.3),
        albu.GaussNoise(p=0.3)
        #albu.GridDistortion(p=.33),
        #albu.HueSaturationValue(p=.33) # not for grey scale
    ], p=p)

In [9]:
#plt.imshow(x)

In [10]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        #print(img.shape)
        #if self.train_mode:
        #    augs = get_train_augs()
        #    img = augs(image=img)['image']
        
        #img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=-1)
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic])

    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
    train_df = shuffle(train_df, random_state=1234)
    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [11]:
#train_loader, val_loader = get_train_val_loaders(dev_mode=True)

# model

In [12]:
#import pretrainedmodels

In [13]:
print(pretrainedmodels.model_names)

['fbresnet152', 'bninception', 'resnext101_32x4d', 'resnext101_64x4d', 'inceptionv4', 'inceptionresnetv2', 'alexnet', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'inceptionv3', 'squeezenet1_0', 'squeezenet1_1', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19_bn', 'vgg19', 'nasnetamobile', 'nasnetalarge', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn131', 'dpn107', 'xception', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'cafferesnet101', 'pnasnet5large', 'polynet']


In [14]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [15]:
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=False).cuda()


In [16]:
#model.features(torch.randn((2, 3, 137, 236)).cuda()).size()

In [17]:
#model.last_linear.in_features

In [18]:
MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

class BengaliNet(nn.Module):
    def __init__(self, backbone_name):
        super(BengaliNet, self).__init__()
        self.n_grapheme = 168
        self.n_vowel = 11
        self.n_consonant = 7
        self.backbone_name = backbone_name
        
        self.num_classes = self.n_grapheme + self.n_vowel + self.n_consonant
        
        #self.conv0 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
        
        if self.backbone_name.startswith('efficient'):
            self.backbone = EfficientNet.from_pretrained(self.backbone_name)
            self.fc = nn.Linear(self.backbone._fc.in_features, self.num_classes)
        else:
            self.backbone = pretrainedmodels.__dict__[self.backbone_name](num_classes=1000, pretrained='imagenet')
            self.fc = nn.Linear(self.backbone.last_linear.in_features, self.num_classes)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        
        #self.fix_input_layer()
        
    def fix_input_layer(self):
        if self.backbone_name in ['se_resnext50_32x4d', 'se_resnext101_32x4d', 'se_resnet50', 'senet154', 'se_resnet152', 'nasnetmobile', 'mobilenet', 'nasnetalarge']:
            #self.backbone = eval(backbone_name)()
            #print(self.backbone.layer0.conv1)
            w = self.backbone.layer0.conv1.weight.data
            self.backbone.layer0.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            #self.backbone.layer0.conv1.weight = torch.nn.Parameter(torch.cat((w, w[:, 2, :, :].unsqueeze(1)), dim=1))
            self.backbone.layer0.conv1.weight = torch.nn.Parameter(w[:, 0, :, :].unsqueeze(1))
        
    def logits(self, x):
        x = self.avg_pool(x)
        #x = F.dropout2d(x, 0.2, self.training)
        x = x.view(x.size(0), -1)
        return self.fc(x)
    
    def forward(self, x):
        x = F.interpolate(x, size=(224,224), mode='bilinear', align_corners=False)
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)
        x = torch.cat([x,x,x], 1)
        #x = self.conv0(x)
        #print(x.size())
        if self.backbone_name.startswith('efficient'):
            x = self.backbone.extract_features(x)
        else:
            x = self.backbone.features(x)
        x = self.logits(x)

        return x

In [19]:
MODEL_DIR = './models'
def create_model(args):
    model = BengaliNet(backbone_name=args.backbone)
    model_file = os.path.join(MODEL_DIR, args.backbone, args.ckp_name)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if args.predict and (not os.path.exists(model_file)):
        raise AttributeError('model file does not exist: {}'.format(model_file))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [20]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [21]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [22]:
round(1/9, 6)

0.111111

In [23]:
import numpy as np
import sklearn.metrics
import torch


def macro_recall(pred_y, y, n_grapheme=168, n_vowel=11, n_consonant=7):
    pred_y = torch.split(pred_y, [n_grapheme, n_vowel, n_consonant], dim=1)
    pred_labels = [torch.argmax(py, dim=1).cpu().numpy() for py in pred_y]

    #y = y.cpu().numpy()
    # pred_y = [p.cpu().numpy() for p in pred_y]

    recall_grapheme = sklearn.metrics.recall_score(pred_labels[0], y_grapheme, average='macro')
    recall_vowel = sklearn.metrics.recall_score(pred_labels[1], y_vowel, average='macro')
    recall_consonant = sklearn.metrics.recall_score(pred_labels[2], y_consonant, average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_score = np.average(scores, weights=[2, 1, 1])
    # print(f'recall: grapheme {recall_grapheme}, vowel {recall_vowel}, consonant {recall_consonant}, '
    #       f'total {final_score}, y {y.shape}')
    return final_score

def calc_metrics(preds0, preds1, preds2, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    
    
    return metrics

In [24]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    
    return loss0 + loss1 + loss2 #, loss0.item(), loss1.item(), loss2.item()

In [25]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2 = 0., 0., 0.
    preds0, preds1,preds2 = [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs = model(x)
            outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    
    return metrics
            

In [26]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [27]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [28]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [29]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [30]:
np.random.random()

0.966877996778257

In [31]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam

In [32]:
def train(args):
    global model

    if args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-5)
    elif args.optim == 'RAdam':
        optimizer = RAdam(model.parameters(), lr=args.lr)
    elif args.optim == 'Over9000':
        optimizer = Over9000(model.parameters(), lr=args.lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=0.)

    if args.lrs == 'plateau':
        lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=args.factor, patience=args.patience, min_lr=args.min_lr)
    else:
        lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, args.t_max, eta_min=args.min_lr)
        
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)

    best_metrics = 0.
    best_key = 'recall'
    
    val_metrics = validate(model, val_loader)
    print(val_metrics)
    best_metrics = val_metrics[best_key]
    
    model.train()
    #optimizer.zero_grad()

    #if args.lrs == 'plateau':
    #    lr_scheduler.step(best_metrics)
    #else:
    #    lr_scheduler.step()
    train_iter = 0

    for epoch in range(args.num_epochs):
        train_loss = 0

        current_lr = get_lrs(optimizer)
        bg = time.time()
        for batch_idx, (img, targets) in enumerate(train_loader):
            train_iter += 1
            img, targets  = img.cuda(), targets.cuda()
            #do_mixup = False #(np.random.random() < 0.4)
            
            #if do_mixup:
            #    img, targets = mixup(img, targets)
            batch_size = img.size(0)
          
            
            
            #if do_mixup:
            #    loss = mixup_criterion(outputs, targets)
            #else:
            #    loss = criterion(outputs, targets)
            r = np.random.rand()
            #if args.beta > 0 and r < args.cutmix_prob:
            if r < 0.5:
                # generate mixed sample
                lam = np.random.beta(args.beta, args.beta)
                rand_index = torch.randperm(img.size()[0]).cuda()
                target_a = targets
                target_b = targets[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
                img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
                # compute output
                outputs = model(img)
                loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            else:
                img, targets = mixup(img, targets)
                outputs = model(img)
                loss = mixup_criterion(outputs, targets)
                #loss = criterion(outputs, targets)
            
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
            
            #loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            
            #if batch_idx % 4 == 0:
            #    optimizer.step()
            #    optimizer.zero_grad()

            train_loss += loss.item()
            print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
                epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
                loss.item(), train_loss/(batch_idx+1)), end='')

            if train_iter > 0 and train_iter % args.iter_val == 0:
                #outputs = torch.split(outputs, [168, 11, 7], dim=1)
            
                #preds0 = (torch.max(outputs[0], dim=1)[1]).cpu().numpy()
                #preds1 = (torch.max(outputs[1], dim=1)[1]).cpu().numpy()
                #preds2 = (torch.max(outputs[2], dim=1)[1]).cpu().numpy()
                #train_metrics = calc_metrics(preds0, preds1, preds2, targets.cpu().numpy())
                #print('train:', train_metrics)
                #save_model(model, model_file+'_latest')
                val_metrics = validate(model, val_loader)
                print('\nval:', val_metrics)
                
                if val_metrics[best_key] > best_metrics:
                    best_metrics = val_metrics[best_key]
                    save_model(model, model_file)
                    print('** saved')
                
                model.train()
                
                if args.lrs == 'plateau':
                    lr_scheduler.step(best_metrics)
                else:
                    lr_scheduler.step()
                current_lr = get_lrs(optimizer)
        
    

In [46]:
args = Namespace()
args.backbone = 'efficientnet-b5'
args.ckp_name = 'model3_efficientnet-b5_fold2_mixup_cutmix_224.pth'
args.predict = False
args.optim = 'Adam'
args.lr = 3e-4
args.lrs = 'cosine'
args.t_max = 12
args.factor = 0.5
args.patience = 5
args.min_lr = 2e-6
args.iter_val = 200
args.num_epochs = 100000
args.batch_size = 320
args.val_batch_size = 1024

args.beta = 1.0
args.cutmix_prob = 0.5

In [34]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=2)

(200840, 5)
(200840, 32332)
(160678, 5) (40162, 5)


In [47]:
model, model_file = create_model(args)
#if torch.cuda.device_count() > 1:
#    model = nn.DataParallel(model)
model = model.cuda()


Loaded pretrained weights for efficientnet-b5
model file: ./models/efficientnet-b5/model3_efficientnet-b5_fold2_mixup_cutmix_224.pth, exist: True
loading ./models/efficientnet-b5/model3_efficientnet-b5_fold2_mixup_cutmix_224.pth...


In [ ]:
train(args)

{'recall': 0.989304, 'recall_grapheme': 0.985111, 'recall_vowel': 0.994026, 'recall_consonant': 0.992967, 'acc_grapheme': 0.984289, 'acc_vowel': 0.994771, 'acc_consonant': 0.994622, 'loss_grapheme': 0.064851, 'loss_vowel': 0.028989, 'loss_consonant': 0.024527}
    0 | 0.000300 | 064000/160678 | 4.0913 | 2.1350 |
val: {'recall': 0.985135, 'recall_grapheme': 0.97954, 'recall_vowel': 0.991045, 'recall_consonant': 0.990413, 'acc_grapheme': 0.979608, 'acc_vowel': 0.993526, 'acc_consonant': 0.993302, 'loss_grapheme': 0.086374, 'loss_vowel': 0.039131, 'loss_consonant': 0.031923}
    0 | 0.000295 | 128000/160678 | 3.6988 | 2.1949 |
val: {'recall': 0.984218, 'recall_grapheme': 0.979975, 'recall_vowel': 0.99184, 'recall_consonant': 0.985082, 'acc_grapheme': 0.979159, 'acc_vowel': 0.992854, 'acc_consonant': 0.991136, 'loss_grapheme': 0.087321, 'loss_vowel': 0.038468, 'loss_consonant': 0.038352}
    1 | 0.000280 | 031360/160678 | 2.1844 | 2.1429 |
val: {'recall': 0.984711, 'recall_grapheme': 0.978

In [36]:
train(args)

{'recall': 0.987769, 'recall_grapheme': 0.982918, 'recall_vowel': 0.993188, 'recall_consonant': 0.992052, 'acc_grapheme': 0.982035, 'acc_vowel': 0.994335, 'acc_consonant': 0.993689, 'loss_grapheme': 0.230987, 'loss_vowel': 0.15323, 'loss_consonant': 0.099472}
    0 | 0.000100 | 144000/160596 | 1.8490 | 1.9192 |
val: {'recall': 0.988941, 'recall_grapheme': 0.985034, 'recall_vowel': 0.994307, 'recall_consonant': 0.991388, 'acc_grapheme': 0.98442, 'acc_vowel': 0.994658, 'acc_consonant': 0.993738, 'loss_grapheme': 0.183109, 'loss_vowel': 0.106912, 'loss_consonant': 0.085281}
** saved


/mnt/chicm/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:113: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


    1 | 0.000098 | 127440/160596 | 1.7123 | 1.9082 |
val: {'recall': 0.990256, 'recall_grapheme': 0.985826, 'recall_vowel': 0.994998, 'recall_consonant': 0.994374, 'acc_grapheme': 0.986955, 'acc_vowel': 0.995403, 'acc_consonant': 0.994707, 'loss_grapheme': 0.214729, 'loss_vowel': 0.156357, 'loss_consonant': 0.098806}
** saved
    2 | 0.000093 | 110880/160596 | 3.3321 | 1.9592 |
val: {'recall': 0.989953, 'recall_grapheme': 0.986059, 'recall_vowel': 0.995261, 'recall_consonant': 0.992434, 'acc_grapheme': 0.986756, 'acc_vowel': 0.995875, 'acc_consonant': 0.994732, 'loss_grapheme': 0.249035, 'loss_vowel': 0.19251, 'loss_consonant': 0.124437}
    3 | 0.000086 | 094320/160596 | 1.0515 | 2.0535 |
val: {'recall': 0.990025, 'recall_grapheme': 0.985642, 'recall_vowel': 0.99463, 'recall_consonant': 0.994187, 'acc_grapheme': 0.98683, 'acc_vowel': 0.995552, 'acc_consonant': 0.99513, 'loss_grapheme': 0.215131, 'loss_vowel': 0.165944, 'loss_consonant': 0.096077}
    4 | 0.000075 | 077760/160596 | 2.6

   25 | 0.000086 | 018000/160596 | 1.8911 | 2.0103 |
val: {'recall': 0.991798, 'recall_grapheme': 0.988771, 'recall_vowel': 0.995519, 'recall_consonant': 0.994133, 'acc_grapheme': 0.989042, 'acc_vowel': 0.996372, 'acc_consonant': 0.995428, 'loss_grapheme': 0.216286, 'loss_vowel': 0.129529, 'loss_consonant': 0.09244}
   26 | 0.000075 | 001440/160596 | 2.9533 | 2.9026 |
val: {'recall': 0.992637, 'recall_grapheme': 0.988959, 'recall_vowel': 0.996413, 'recall_consonant': 0.99622, 'acc_grapheme': 0.989216, 'acc_vowel': 0.996571, 'acc_consonant': 0.995701, 'loss_grapheme': 0.212417, 'loss_vowel': 0.163295, 'loss_consonant': 0.097383}
** saved
   26 | 0.000063 | 145440/160596 | 2.6632 | 1.9373 |
val: {'recall': 0.992202, 'recall_grapheme': 0.988811, 'recall_vowel': 0.996096, 'recall_consonant': 0.995089, 'acc_grapheme': 0.988048, 'acc_vowel': 0.996546, 'acc_consonant': 0.995776, 'loss_grapheme': 0.218281, 'loss_vowel': 0.173208, 'loss_consonant': 0.099098}
   27 | 0.000051 | 128880/160596 | 2

   48 | 0.000063 | 069120/160596 | 2.7455 | 1.7793 |
val: {'recall': 0.993727, 'recall_grapheme': 0.991098, 'recall_vowel': 0.99662, 'recall_consonant': 0.996093, 'acc_grapheme': 0.990582, 'acc_vowel': 0.996621, 'acc_consonant': 0.996173, 'loss_grapheme': 0.172522, 'loss_vowel': 0.114462, 'loss_consonant': 0.078324}
** saved
   49 | 0.000051 | 052560/160596 | 2.0017 | 1.8439 |
val: {'recall': 0.993, 'recall_grapheme': 0.989735, 'recall_vowel': 0.996357, 'recall_consonant': 0.996175, 'acc_grapheme': 0.989464, 'acc_vowel': 0.996795, 'acc_consonant': 0.99585, 'loss_grapheme': 0.216581, 'loss_vowel': 0.155391, 'loss_consonant': 0.094304}
   50 | 0.000038 | 036000/160596 | 2.8190 | 1.7392 |
val: {'recall': 0.993665, 'recall_grapheme': 0.990976, 'recall_vowel': 0.99635, 'recall_consonant': 0.996357, 'acc_grapheme': 0.990707, 'acc_vowel': 0.996894, 'acc_consonant': 0.996447, 'loss_grapheme': 0.202577, 'loss_vowel': 0.141524, 'loss_consonant': 0.08619}
   51 | 0.000026 | 019440/160596 | 0.7174

...

  188 | 0.000038 | 054720/160596 | 2.4831 | 1.8379 |
val: {'recall': 0.99382, 'recall_grapheme': 0.991322, 'recall_vowel': 0.995696, 'recall_consonant': 0.996938, 'acc_grapheme': 0.991229, 'acc_vowel': 0.996422, 'acc_consonant': 0.996571, 'loss_grapheme': 0.185547, 'loss_vowel': 0.134004, 'loss_consonant': 0.088937}
  189 | 0.000051 | 038160/160596 | 1.6782 | 1.6708 |
val: {'recall': 0.994778, 'recall_grapheme': 0.993, 'recall_vowel': 0.996225, 'recall_consonant': 0.996887, 'acc_grapheme': 0.992322, 'acc_vowel': 0.997267, 'acc_consonant': 0.997043, 'loss_grapheme': 0.196368, 'loss_vowel': 0.10847, 'loss_consonant': 0.071526}
  190 | 0.000063 | 021600/160596 | 2.3653 | 1.6604 |
val: {'recall': 0.992898, 'recall_grapheme': 0.990522, 'recall_vowel': 0.99579, 'recall_consonant': 0.99476, 'acc_grapheme': 0.989688, 'acc_vowel': 0.996944, 'acc_consonant': 0.996099, 'loss_grapheme': 0.243178, 'loss_vowel': 0.129244, 'loss_consonant': 0.087587}
  191 | 0.000075 | 005040/160596 | 0.5627 | 1.7813

  211 | 0.000063 | 105840/160596 | 3.5722 | 1.6140 |
val: {'recall': 0.994234, 'recall_grapheme': 0.991927, 'recall_vowel': 0.996679, 'recall_consonant': 0.996401, 'acc_grapheme': 0.991055, 'acc_vowel': 0.997192, 'acc_consonant': 0.996795, 'loss_grapheme': 0.229229, 'loss_vowel': 0.119446, 'loss_consonant': 0.076131}
  212 | 0.000075 | 089280/160596 | 2.5102 | 1.6650 |
val: {'recall': 0.994533, 'recall_grapheme': 0.99203, 'recall_vowel': 0.996714, 'recall_consonant': 0.997355, 'acc_grapheme': 0.991899, 'acc_vowel': 0.997515, 'acc_consonant': 0.997391, 'loss_grapheme': 0.254168, 'loss_vowel': 0.130995, 'loss_consonant': 0.084437}
  213 | 0.000086 | 072720/160596 | 1.6722 | 1.7846 |
val: {'recall': 0.992733, 'recall_grapheme': 0.989459, 'recall_vowel': 0.995626, 'recall_consonant': 0.996387, 'acc_grapheme': 0.989116, 'acc_vowel': 0.996148, 'acc_consonant': 0.99595, 'loss_grapheme': 0.168562, 'loss_vowel': 0.100059, 'loss_consonant': 0.07559}
  214 | 0.000093 | 056160/160596 | 1.7666 | 1.

  234 | 0.000086 | 156960/160596 | 2.3966 | 1.6234 |
val: {'recall': 0.992841, 'recall_grapheme': 0.990413, 'recall_vowel': 0.995874, 'recall_consonant': 0.994664, 'acc_grapheme': 0.990458, 'acc_vowel': 0.996397, 'acc_consonant': 0.996347, 'loss_grapheme': 0.20179, 'loss_vowel': 0.104209, 'loss_consonant': 0.06583}
  235 | 0.000093 | 140400/160596 | 1.1907 | 1.7767 |
val: {'recall': 0.993624, 'recall_grapheme': 0.991629, 'recall_vowel': 0.996067, 'recall_consonant': 0.995173, 'acc_grapheme': 0.991353, 'acc_vowel': 0.996844, 'acc_consonant': 0.996944, 'loss_grapheme': 0.230878, 'loss_vowel': 0.122597, 'loss_consonant': 0.082995}
  236 | 0.000098 | 123840/160596 | 1.5327 | 1.7184 |
val: {'recall': 0.992581, 'recall_grapheme': 0.990146, 'recall_vowel': 0.995158, 'recall_consonant': 0.994876, 'acc_grapheme': 0.990508, 'acc_vowel': 0.996248, 'acc_consonant': 0.9959, 'loss_grapheme': 0.198729, 'loss_vowel': 0.106843, 'loss_consonant': 0.072458}
  237 | 0.000100 | 107280/160596 | 1.0317 | 1.8

  258 | 0.000098 | 047520/160596 | 0.9052 | 1.7931 |
val: {'recall': 0.994441, 'recall_grapheme': 0.992623, 'recall_vowel': 0.996707, 'recall_consonant': 0.995808, 'acc_grapheme': 0.9918, 'acc_vowel': 0.997242, 'acc_consonant': 0.996596, 'loss_grapheme': 0.195736, 'loss_vowel': 0.104524, 'loss_consonant': 0.070656}
  259 | 0.000100 | 030960/160596 | 2.3566 | 1.6954 |
val: {'recall': 0.992888, 'recall_grapheme': 0.989383, 'recall_vowel': 0.996531, 'recall_consonant': 0.996253, 'acc_grapheme': 0.990235, 'acc_vowel': 0.996819, 'acc_consonant': 0.996198, 'loss_grapheme': 0.223854, 'loss_vowel': 0.133408, 'loss_consonant': 0.082446}
  260 | 0.000098 | 014400/160596 | 2.7934 | 1.6074 |
val: {'recall': 0.994105, 'recall_grapheme': 0.992519, 'recall_vowel': 0.995995, 'recall_consonant': 0.995389, 'acc_grapheme': 0.991601, 'acc_vowel': 0.99672, 'acc_consonant': 0.997018, 'loss_grapheme': 0.272782, 'loss_vowel': 0.120502, 'loss_consonant': 0.082534}
  260 | 0.000093 | 158400/160596 | 0.5477 | 1.

KeyboardInterrupt: 

In [ ]:
#save_model(model, model_file)

In [45]:
del model